In [103]:
from re import match
import pandas as pd
import numpy as np
import glob
import os
import pickle

def load_and_combine_result_data_from_csv(results_path) -> pd.DataFrame:
    """
    Load all individual season and league result data and combine into one data frame.
    """

    all_files = glob.glob(os.path.join(results_path, "*"))
    df_list = []

    for path in all_files:
        all_csv_files = glob.glob(os.path.join(path, "*.csv"))
        df_name = (path.split("/")[9])
        df_list.append(f"{df_name}_df")
        globals()[f"{df_name}_df"] = pd.concat((pd.read_csv(f) for f in all_csv_files), ignore_index=True)

    combined_results_df = pd.concat((globals()[df] for df in df_list), ignore_index=True)

    return combined_results_df

def load_match_and_team_info_from_csv(info_path, team_path) -> pd.DataFrame:
    """
    Load Pandas DF from CSV Files
    """
    match_info_df = pd.read_csv(info_path)
    team_info_df = pd.read_csv(team_path)

    return match_info_df, team_info_df

def _clean_results_data(combined_results_df) -> pd.DataFrame:
    """
    Function to clean results data.
    """
    
    combined_results_unnamed_removed_df = _remove_unnamed_values(combined_results_df)
    combined_results_remove_invalid_scores_df = _remove_invalid_scores(combined_results_unnamed_removed_df)
    combined_results_standarised_match_links_df = _standardise_match_links(combined_results_remove_invalid_scores_df)
    combined_results_data_types_df = _convert_results_data_types(combined_results_standarised_match_links_df)

    return combined_results_data_types_df


def _convert_results_data_types(combined_results_df):
    """
    Function to update data types of columns to the erquired formats.
    """
    # Update data types
    combined_results_df["Home_Team"] = combined_results_df["Home_Team"].astype('category')
    combined_results_df["Away_Team"] = combined_results_df["Away_Team"].astype('category')
    combined_results_df["Result"] = combined_results_df["Result"].astype('string')
    combined_results_df["Season"] = combined_results_df["Season"].astype('int')
    combined_results_df["Round"] = combined_results_df["Round"].astype('int')
    combined_results_df["League"] = combined_results_df["League"].astype('string')
    combined_results_df["Link"] = combined_results_df["Link"].astype('string')

    return combined_results_df

def _remove_unnamed_values(combined_results_df):
    """
    Function to remove presence of unknown variable
    """
    # remove unnamed values
    combined_results_df.drop("Unnamed: 0", axis = 1, inplace=True)
    return combined_results_df

def _remove_invalid_scores(combined_results_df):
    '''_remove_invalid_scores 
    Function to remove invalid scores.

    A number of scores are in invlid formats. This function modifies wha is known and removes what isn't.

    Arguments:
        combined_results_df -- Combined results dataframe

    Returns:
        updated combined results dataframe
    '''

    # identify and remove invalid scores

    invalid_score_1 = (combined_results_df.loc[: , "Result"].str.contains(":"))
    invalid_score_2 = (combined_results_df.loc[: , "Result"].str.len() > 5)
    invalid_score_3 = (combined_results_df.loc[: , "Result"].str.len() < 2)

    combined_results_df.replace("1 (0-0) 1", np.NaN, inplace=True)
    combined_results_df.replace("0 (0-0) 0", "0-0", inplace=True)
    combined_results_df.replace("3 (3-2) 2", "3-2", inplace=True)
    combined_results_df.replace("0 (0-1) 1", "0-1", inplace=True)

    combined_results_df[invalid_score_1] = np.NaN
    combined_results_df[invalid_score_2] = np.NaN
    combined_results_df[invalid_score_3] = np.NaN

    combined_results_df.dropna(subset=["Result"], inplace = True)
    
    return combined_results_df

def _standardise_match_links(combined_results_df):
    '''_standardise_match_links 
    Function to standardise the links to enable merging between datasets.
    '''
    combined_results_df["Link"] = combined_results_df["Link"].apply(lambda x: "".join(x.replace(x.split("/")[6], "")) + x.split("/")[6][0:4])
    
    return combined_results_df

def _clean_match_info_data(match_info_df) -> pd.DataFrame:
    '''clean_match_info_data 
    

    Returns:
        _description_
    '''
    match_info_card_df = _replace_missing_card_data(match_info_df)
    match_info_data_types_converted_df = _convert_match_info_data_types(match_info_card_df)
    match_info_standardised_df = _standardise_referee_and_links(match_info_data_types_converted_df)

    return match_info_standardised_df

def _replace_missing_card_data(match_info_df):
    '''_replace_missing_card_data _summary_

    Arguments:
        match_info_df -- _description_
    '''
    match_info_df["Home_Yellow"].replace(np.NaN, 0, inplace=True)
    match_info_df["Home_Red"].replace(np.NaN, 0, inplace=True)
    match_info_df["Away_Yellow"].replace(np.NaN, 0, inplace=True)
    match_info_df["Away_Red"].replace(np.NaN, 0, inplace=True)

    return match_info_df

def _convert_match_info_data_types(match_info_df):
    '''_convert_data_types 

    Arguments:
        match_info_df -- _description_
    '''
    match_info_df["Date_New"] = match_info_df["Date_New"].astype('datetime64')
    match_info_df["Referee"] = match_info_df["Referee"].astype('string')
    match_info_df["Home_Yellow"] = match_info_df["Home_Yellow"].astype('int')
    match_info_df["Home_Red"] = match_info_df["Home_Red"].astype('int')
    match_info_df["Away_Yellow"] = match_info_df["Away_Yellow"].astype('int')
    match_info_df["Away_Red"] = match_info_df["Away_Red"].astype('int')
    match_info_df["Link"] = match_info_df["Link"].astype('string')

    return match_info_df

def _standardise_referee_and_links(match_info_df):
    '''_standardise_referee_and_links _summary_

    Arguments:
        match_info_df -- _description_
    '''
    match_info_df["Referee"] = match_info_df["Referee"].apply(lambda x: (x.replace("\r\n", "")).replace("Referee: ", ""))
    match_info_df["Link"] = match_info_df["Link"].apply(lambda x: "https://www.besoccer.com" + x)

    return match_info_df

def _clean_team_info_data(team_info_df):
    '''clean_team_info_data 

    Arguments:
        team_info_df -- _description_
    '''
    team_info_df["Capacity"] = team_info_df["Capacity"].apply(lambda x: x.replace(",", ""))
    team_info_df["Capacity"] = team_info_df["Capacity"].astype('int')

    # rename column in team info

    team_info_df = team_info_df.rename(columns = {'Team' : 'Home_Team'})

    return team_info_df

def clean_all_dataframes(combined_results_df, match_info_df, team_info_df):
    combined_results_cleaned_df = _clean_results_data(combined_results_df)
    match_info_cleaned_df = _clean_match_info_data(match_info_df)
    team_info_cleaned_df = _clean_team_info_data(team_info_df)

    return combined_results_cleaned_df, match_info_cleaned_df, team_info_cleaned_df

def add_features_to_results_df(combined_results_cleaned_df):
    '''add_features_to_results_df _summary_

    Arguments:
        combined_results_cleaned_df -- _description_
    '''
    combined_results_cleaned_df["Home_Goals"] = combined_results_cleaned_df["Result"].apply(lambda x: x.split("-")[0])
    combined_results_cleaned_df["Away_Goals"] = combined_results_cleaned_df["Result"].apply(lambda x: x.split("-")[1])
    combined_results_cleaned_df["Home_Win"] = combined_results_cleaned_df["Home_Goals"] > combined_results_cleaned_df["Away_Goals"]
    combined_results_cleaned_df["Away_Win"] = combined_results_cleaned_df["Home_Goals"] < combined_results_cleaned_df["Away_Goals"]

    combined_results_cleaned_df["Home_Goals"] = combined_results_cleaned_df["Home_Goals"].astype('int')
    combined_results_cleaned_df["Away_Goals"] = combined_results_cleaned_df["Away_Goals"].astype('int')

    return combined_results_cleaned_df

def combine_dataframes(combined_results_cleaned_df, match_info_cleaned_df, team_info_cleaned_df) -> pd.DataFrame:
    '''combine_dataframes _summary_

    Arguments:
        combined_results_cleaned_df -- _description_
        match_info_cleaned_df -- _description_
        team_info_cleaned_df -- _description_

    Returns:
        _description_
    '''
    result_match_df = pd.merge(combined_results_cleaned_df, match_info_cleaned_df, how='inner', on="Link")
    combined_df = pd.merge(result_match_df, team_info_cleaned_df, how='inner', on="Home_Team")

    return combined_df

def _load_ELO_pickle_data() -> pd.DataFrame:
    ELO_dict = pickle.load(open('2. Feature Engineering/elo_dict.pkl', 'rb'))
    ELO_df = pd.DataFrame(ELO_dict.items(), columns=["Link", "ELO_dict"])

    return ELO_df

def _clean_EDA_ELO_data(ELO_df):
    ELO_df["ELO_home"] = (ELO_df["ELO_dict"]).apply(lambda x: x['Elo_home'])
    ELO_df["ELO_away"] = (ELO_df["ELO_dict"]).apply(lambda x: x['Elo_away'])
    ELO_df.drop(columns="ELO_dict", inplace=True)
    ELO_df["Link"] = ELO_df["Link"].apply(lambda x: "".join(x.replace(x.split("/")[6], "")) + x.split("/")[6][0:4])

    return ELO_df

def _merge_ELO_combined_df(combined_df, ELO_df):

    combined_ELO_df = pd.merge(combined_df, ELO_df, how='inner', on="Link")

    return combined_ELO_df

def load_clean_merge_ELO_data(combined_df):

    ELO_df = _load_ELO_pickle_data()
    ELO_cleaned_df = _clean_EDA_ELO_data(ELO_df)
    combined_ELO_df = _merge_ELO_combined_df(combined_df, ELO_cleaned_df)

    return combined_ELO_df




In [104]:
# SetUp Visual

pd.set_option('display.max_columns', None)

# Paths

results_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Results'
info_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Other/Match_Info.csv'
team_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Other/Team_Info.csv'

## Caller Functions

combined_results_df = load_and_combine_result_data_from_csv(results_path) 
match_info_df, team_info_df = load_match_and_team_info_from_csv(info_path, team_path)
combined_results_cleaned_df, match_info_cleaned_df, team_info_cleaned_df = clean_all_dataframes(combined_results_df, match_info_df, team_info_df)
combined_results_features_df = add_features_to_results_df(combined_results_cleaned_df)
combined_df = combine_dataframes(combined_results_cleaned_df, match_info_cleaned_df, team_info_cleaned_df)
combined_ELO_df = load_clean_merge_ELO_data(combined_df)

Milestone 3 - Feature Engineering

In [203]:

def create_new_feature_columns(combined_elo_df):
    '''create_new_feature_columns 
    Function to create columns filled with zeros for each required feature

    Arguments:
        combined_local_elo_df -- _description_
    '''
    combined_elo_df["total_goals_scored_so_far"] = 0
    combined_elo_df["total_goals_conceeded_so_far"] = 0
    combined_elo_df["largest_win_streak_so_far"] = 0
    combined_elo_df["largest_loss_streak_so_far"] = 0
    combined_elo_df["total_points_so_far"] = 0
    combined_elo_df["longest_goal_drought_so_far"] = 0
    combined_elo_df["total_wins_so_far"] = 0

    return combined_elo_df

def group_by_league_df(combined_elo_df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to group full dataframe into only the required league

    """

    league = 'premier_league'
    grouped_df = combined_elo_df.groupby("League")
    league_df = grouped_df.get_group(league)

    return league_df


def group_by_season_df(league_df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to group full dataframe into seasons

    """

    season = 2019.0
    grouped_df = league_df.groupby("Season")
    season_df = grouped_df.get_group(season)
    # Order season by round
    season_df = season_df.sort_values(by=['Round'], ascending=[True])

    return season_df


def create_summary_template(season_df):
    '''create_summary_template 
    Function to create auxiliary template summary per season

    Arguments:
        season_df -- _description_

    Returns:
        _description_
    '''

    summary_df_template = pd.DataFrame(columns=
    [
        'team_name', 'season', 'league', 'total_goals_scored_so_far', 'total_goals_conceeded_so_far', 
        'largest_win_streak_so_far', 'largest_loss_streak_so_far', 'total_points_so_far', 'longest_goal_drought_so_far', 
        'total_wins_so_far'
        ])
    team_group = season_df.groupby("Home_Team")

    for home_team_name in season_df["Home_Team"].unique():
        team_df = team_group.get_group(home_team_name)     
        team_name = home_team_name
        season = team_df["Season"].unique()[0]
        league = team_df["League"].unique()[0]

        team_stats_dict = {
            'team_name': team_name, 'season': season, 
            'league': league, 'total_goals_scored_so_far': 0, 
            'total_goals_conceeded_so_far': 0, 'largest_win_streak_so_far': 0,
            'largest_loss_streak_so_far': 0, 'total_points_so_far': 0, 
            'longest_goal_drought_so_far': 0, 'total_wins_so_far': 0
            }
        team_stats_dict_df = pd.DataFrame([team_stats_dict])
        summary_df_template = pd.concat([summary_df_template, team_stats_dict_df], ignore_index=True)


    print("Summary Template Complete")
    return summary_df_template

def populate_season_stats(season_df, summary_df_template):
    '''populate_season_stats 
    Function to add to season templte summary per round.

    Arguments:
        season_df -- _description_
        summary_df_template -- _description_

    Returns:
        _description_
    '''


    round_df_grouped = season_df.groupby("Round")

    for round in season_df["Round"].unique():
        round_df = round_df_grouped.get_group(round)
        for index, row in round_df.iterrows():

            home_team_name = row["Home_Team"]
            away_team_name = row["Away_Team"]
            season = row["Season"]
            league = row["League"]
            home_goals_scored = row["Home_Goals"]
            away_goals_scored = row["Away_Goals"]
            home_win = row["Home_Win"]
            away_win = row["Away_Win"]

            # Home Team
            summary_df_template = _goals_and_points_totaller(summary_df_template, home_team_name, home_goals_scored, away_goals_scored, season, league)
            # Away Team
            summary_df_template = _goals_and_points_totaller(summary_df_template, away_team_name, away_goals_scored, home_goals_scored, season, league)
        
        for team in list(summary_df_template["team_name"]):
            season_df = _add_feature_total_to_df("total_goals_scored_so_far", team, season, league, season_df, summary_df_template, round)
            season_df = _add_feature_total_to_df("total_goals_conceeded_so_far", team, season, league, season_df, summary_df_template, round)
            season_df = _add_feature_total_to_df("total_points_so_far", team, season, league, season_df, summary_df_template, round)
            season_df = _add_feature_total_to_df("total_wins_so_far", team, season, league, season_df, summary_df_template, round)

    print(f"Summary DF Template Populated for Season {season}")

    return summary_df_template, season_df

def _goals_and_points_totaller(summary_df_template, team, goal_option_1, goal_option_2, season, league):

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.season == season) & (
    summary_df_template.league == league)), "total_goals_scored_so_far"] += goal_option_1

    summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_goals_conceeded_so_far"] += goal_option_2
        
    if goal_option_1 > goal_option_2:

        summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_points_so_far"] += 3
        summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_wins_so_far"] += 1
    
    if goal_option_1 == goal_option_2:

        summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_points_so_far"] += 1
    
    return summary_df_template

def _add_feature_total_to_df(feature, team, season, league, season_df, summary_df_template, round):
    
    season_df.loc[((season_df.Home_Team == team) & (
            season_df.Season == season) & (season_df.League == league) & (
                season_df.Round == round)), feature] = summary_df_template.loc[((
                    summary_df_template.team_name == team) & (summary_df_template.season == season) & (
                        summary_df_template.league == league)), feature].values[0]
    return season_df


In [204]:
combined_elo_with_features_df = create_new_feature_columns(combined_ELO_df)
league_df = group_by_league_df(combined_elo_with_features_df)
season_df = group_by_season_df(league_df)
summary_df_template = create_summary_template(season_df)
summary_df_populated, test_df = populate_season_stats(season_df, summary_df_template)




Summary Template Complete
Summary DF Template Populated for Season 2019


In [161]:
team = "Watford"
season = 2019
league = "premier_league"
round = 1
# summary df template
summary_df_populated.loc[((summary_df_populated.team_name == team) & (summary_df_populated.season == season) & (summary_df_populated.league == league)), "total_goals_scored"].values[0]
# combined elo option
#season_df.loc[((season_df.Home_Team == team) & (season_df.Season == season) & (season_df.League == league) & (season_df.Round == round)), "total_goals_scored_so_far"]

52

In [205]:
test_df

,Home_Team,Away_Team,Result,Link,Season,Round,League,Home_Goals,Away_Goals,Home_Win,Away_Win,Date_New,Referee,Home_Yellow,Home_Red,Away_Yellow,Away_Red,City,Country,Stadium,Capacity,Pitch,ELO_home,ELO_away,total_goals_scored_so_far,total_goals_conceeded_so_far,largest_win_streak_so_far,largest_loss_streak_so_far,total_points_so_far,longest_goal_drought_so_far,total_wins_so_far
353,Watford,Brighton & Hove Albion,2-0,https://www.besoccer.com/match/watford-fc/brig...,2019,1,premier_league,2,0,True,False,2018-08-11 16:00:00,Moss J.,2,0,2,0,Watford,England,Vicarage Road Stadium,19920,Natural,72.0,66.0,2,0,0,0,3,0,1
113097,Man. Utd,Leicester,2-1,https://www.besoccer.com/match/manchester-unit...,2019,1,premier_league,2,1,True,False,2018-08-10 21:00:00,Marriner A.,2,0,1,0,Mánchester,England,Old Trafford,76212,Natural,95.0,82.0,2,1,0,0,3,0,1
2048,Huddersfield Town,Chelsea,0-3,https://www.besoccer.com/match/huddersfield-to...,2019,1,premier_league,0,3,False,True,2018-08-11 16:00:00,Kavanagh C.,2,0,1,0,Huddersfield,England,The John Smith's Stadium,24554,Natural,61.0,93.0,0,3,0,0,0,0,0
6041,Wolves,Everton,2-2,https://www.besoccer.com/match/wolverhampton/e...,2019,1,premier_league,2,2,False,False,2018-08-11 18:30:00,Pawson C.,0,0,1,1,Wolverhampton,England,Molineux Stadium,29195,Natural,69.0,83.0,2,2,0,0,1,0,0
111878,Arsenal,Man. City,0-2,https://www.besoccer.com/match/arsenal/manches...,2019,1,premier_league,0,2,False,True,2018-08-12 17:00:00,Oliver M.,2,0,2,0,Londres,England,Emirates Stadium,60355,Natural,92.0,96.0,0,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,Watford,West Ham,1-4,https://www.besoccer.com/match/watford-fc/west...,2019,38,premier_league,1,4,False,True,2019-05-12 16:00:00,Kavanagh C.,1,1,0,0,Watford,England,Vicarage Road Stadium,19920,Natural,75.0,75.0,52,59,0,0,49,0,14
10179,Crystal Palace,AFC Bournemouth,5-3,https://www.besoccer.com/match/crystal-palace-...,2019,38,premier_league,5,3,True,False,2019-05-12 16:00:00,East R.,3,0,0,0,Londres,England,Selhurst Park,26309,Natural,77.0,72.0,50,50,0,0,49,0,14
113115,Man. Utd,Cardiff City,0-2,https://www.besoccer.com/match/manchester-unit...,2019,38,premier_league,0,2,False,True,2019-05-12 16:00:00,Moss J.,3,0,3,0,Mánchester,England,Old Trafford,76212,Natural,93.0,61.0,63,51,0,0,66,0,19
13687,Burnley,Arsenal,1-3,https://www.besoccer.com/match/burnley-fc/arse...,2019,38,premier_league,1,3,False,True,2019-05-12 16:00:00,Dean M.,5,0,1,0,Burnley,England,Turf Moor,22546,Natural,73.0,91.0,42,67,0,0,37,0,10


In [196]:
summary_df_template

,team_name,season,league,total_goals_scored_so_far,total_goals_conceeded_so_far,largest_win_streak_so_far,largest_loss_streak_so_far,total_points_so_far,longest_goal_drought_so_far,total_wins_so_far
0,Watford,2019,premier_league,2,0,0,0,3,0,1
1,Man. Utd,2019,premier_league,2,1,0,0,3,0,1
2,Huddersfield Town,2019,premier_league,0,3,0,0,0,0,0
3,Wolves,2019,premier_league,2,2,0,0,1,0,0
4,Arsenal,2019,premier_league,0,2,0,0,0,0,0
5,Liverpool,2019,premier_league,4,0,0,0,3,0,1
6,Southampton,2019,premier_league,0,0,0,0,1,0,0
7,AFC Bournemouth,2019,premier_league,2,0,0,0,3,0,1
8,Newcastle,2019,premier_league,1,2,0,0,0,0,0
9,Fulham,2019,premier_league,0,2,0,0,0,0,0
